In [1]:
from deepface import DeepFace
#liveness
import tensorflow as tf


2023-12-05 23:41:09.837493: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import cv2, numpy, datetime, pygame, keyboard
import xlwings as xw
import os

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
model = 'liveness.model'
model = tf.keras.models.load_model(model)
    

In [4]:
#xlwings
#initializing workbook and worksheet
workbook = xw.Book('attendance.xlsx')
sheet_name = datetime.date.today().isoformat()

try:
    worksheet = workbook.sheets(sheet_name)
except:
    worksheet = workbook.sheets.add(sheet_name)

In [5]:
#xlwings

worksheet.range('A1').value = 'STUDENT'
worksheet.range('B1').value = 'DATE'
worksheet.range('C1').value = 'TIME'

students=[]

In [6]:
s= 2

i = 0

entered = False
al_entered = False

In [ ]:
cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

cap.set(cv2.CAP_PROP_FPS, 30)


#pygame

pygame.init()

pg_txt = (39,112,50)
pg_bg = (154,179,157)

font = cv2.FONT_HERSHEY_SIMPLEX
pygame_font = pygame.font.Font('freesansbold.ttf',32)

screen = pygame.display.set_mode((960,540))

background = pygame.image.load('background_image.jpg')
background = pygame.transform.rotozoom(background,0,0.8)
screen.blit(background,(0,0))


#day today
t0 = datetime.date.today()
t0 = t0.day
start_time = None
running = True

while running:
    
    moment = datetime.datetime.now()
    hour = moment.hour
    minute = moment.minute
    day = moment.day
    month = moment.month
    year = moment.year
    
    date = f"{day}-{month}-{year}"
    time = f"{hour}:{minute}"
        
    screen.blit(background,(0,0))
    
    #check 
    
    if day!=t0:
        t0 = day
        
        worksheet = workbook.sheets.add(date)

        worksheet.range('A1').value = 'STUDENT'
        worksheet.range('B1').value = 'DATE'
        worksheet.range('C1').value = 'TIME'
        students = []
        s=2
        i = 0
        entered = False
        al_entered = False
    
    
    
    
    
    state, frame = cap.read()
    
    if state!=True:
        break
        
    res  = DeepFace.find(frame, db_path='./Database/', enforce_detection=False, model_name='VGG-Face')
    
    if len(res[0]['identity'])>0:
        file_path = res[0]['identity'][0]
        parts = file_path.split('/')
        name = parts[-2] 
        
#         name = res[0]['identity'][0]
# #         .split('/')[1].split('\\')[1]
        xmin = int(res[0]['source_x'][0])
        ymin = int(res[0]['source_y'][0])
        
        w = res[0]['source_w'][0]
        h = res[0]['source_h'][0]
        
        xmax = int(xmin + w)
        ymax = int(ymin + h)
        
        #liveness
        
        img = frame[ymin:ymax, xmin:xmax]
        img = cv2.resize(img, (32,32))
        img = img.astype('float')/255.0
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = numpy.expand_dims(img, axis=0)
        
        liveness = model.predict(img)
        
        liveness = liveness[0].argmax()
        
        
        

        
        if liveness==1:
            cv2.rectangle(frame, (xmin,ymin),(xmax,ymax), (0,255,0),1)
            cv2.rectangle(frame, (xmin, ymin-25),(xmax, ymin),(255,255,255),-1)
            cv2.putText(frame, name, (xmin,ymin), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0),2,cv2.LINE_AA)
            
            n_txt = pygame_font.render(name, True, pg_txt, pg_bg)
            n_txtrect = n_txt.get_rect()
            n_txtrect.center =(960-200, 540//2)
            
            d_txt = pygame_font.render(date, True, pg_txt, pg_bg)
            d_txtrect = d_txt.get_rect()
            d_txtrect.center= (960-200, (540//2)+40)
            
            t_txt = pygame_font.render(time, True, pg_txt, pg_bg)
            t_txtrect = d_txt.get_rect()
            t_txtrect.center = (960-200, (540//2)+80)         
            
            
            screen.blit(n_txt, n_txtrect)
            screen.blit(d_txt, d_txtrect)
            screen.blit(t_txt, t_txtrect)
            # If face detected, start or update the timer
            if start_time is None:
                start_time = moment
            elif (moment - start_time).seconds > 10:
                # If face is detected for more than 1 minute, update attendance
                if name not in students:
                    worksheet.range(f'A{s}').value = name
                    worksheet.range(f'B{s}').value = date
                    worksheet.range(f'c{s}').value = time
                    students.append(name)
                    s+=1
                start_time = None

            
#             if keyboard.is_pressed('enter'):
                
#                 if name not in students:
                    
#                     worksheet.range(f'A{s}').value = name
#                     worksheet.range(f'B{s}').value = date
#                     worksheet.range(f'c{s}').value = time
#                     students.append(name)
#                     s+=1
#                     entered =True
#                 else:
#                     al_entered = True
                    
            if i<30 and entered==True:
                e_txt = pygame_font.render('entered', True, pg_txt, pg_bg)
                e_txtrect = e_txt.get_rect()
                e_txtrect.center = (960-200, (540//2)+120)
                screen.blit(e_txt, e_txtrect)
                i+=1

            elif i<30 and al_entered==True:
                ae_txt = pygame_font.render('already entered', True, pg_txt, pg_bg)
                ae_txtrect = ae_txt.get_rect()
                ae_txtrect.center = (960-200, (540//2)+120)
                screen.blit(ae_txt, e_txtrect)    
                i+=1

            else:
                entered=False
                al_entered = False
                i = 0

    
    
    image = pygame.image.frombuffer(frame.tostring(), frame.shape[1::-1], 'BGR')
    image = pygame.transform.rotozoom(image, 0 , 0.8)
    #round corners--
    size = image.get_size()
    rect_img = pygame.Surface(size, pygame.SRCALPHA)
    pygame.draw.rect(rect_img, (255,255,255), (0,0,*size), border_radius=10)
    
    image = image.copy().convert_alpha()
    image.blit(rect_img, (0,0), None, pygame.BLEND_RGB_MIN)
    #round corner ---
    
    screen.blit(image, (20,80))
    
    
    pygame.display.update()
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    #cv2.imshow('attendance', frame)
    
    #c = cv2.waitKey(1)
    
    #if c == ord('q'):
        #break
        
cap.release()
pygame.quit()
#cv2.destroyAllWindows()

2023-12-05 23:41:13.228 python[9086:732958] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


There are  12  representations found in  representations_vgg_face.pkl
find function lasts  1.9093530178070068  seconds
1/1 [==============================] - 0s 108ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.2427520751953125  seconds
1/1 [==============================] - 0s 18ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.22679615020751953  seconds
1/1 [==============================] - 0s 18ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.23885488510131836  seconds
1/1 [==============================] - 0s 17ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.23514914512634277  seconds
1/1 [==============================] - 0s 16ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.23668575286865234  seconds
1/1 [================

find function lasts  0.2478468418121338  seconds
1/1 [==============================] - 0s 17ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.24206328392028809  seconds
1/1 [==============================] - 0s 17ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.2362980842590332  seconds
1/1 [==============================] - 0s 16ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.22990012168884277  seconds
1/1 [==============================] - 0s 16ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.2268199920654297  seconds
1/1 [==============================] - 0s 17ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.24080705642700195  seconds
1/1 [==============================] - 0s 16ms/step
There are  12  representations found in  

find function lasts  0.229050874710083  seconds
1/1 [==============================] - 0s 17ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.22972488403320312  seconds
1/1 [==============================] - 0s 17ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.22611308097839355  seconds
1/1 [==============================] - 0s 17ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.2354421615600586  seconds
1/1 [==============================] - 0s 17ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.22817015647888184  seconds
1/1 [==============================] - 0s 17ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.23394989967346191  seconds
1/1 [==============================] - 0s 17ms/step
There are  12  representations found in  

find function lasts  0.25942516326904297  seconds
1/1 [==============================] - 0s 18ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.2568800449371338  seconds
1/1 [==============================] - 0s 17ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.2525670528411865  seconds
1/1 [==============================] - 0s 18ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.250823974609375  seconds
1/1 [==============================] - 0s 17ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.25188302993774414  seconds
1/1 [==============================] - 0s 17ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.2583811283111572  seconds
1/1 [==============================] - 0s 17ms/step
There are  12  representations found in  re

...

find function lasts  0.44052886962890625  seconds
1/1 [==============================] - 0s 33ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.40491819381713867  seconds
1/1 [==============================] - 0s 85ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.42380785942077637  seconds
1/1 [==============================] - 0s 76ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.5217037200927734  seconds
1/1 [==============================] - 0s 29ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.7122681140899658  seconds
1/1 [==============================] - 0s 26ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.4384729862213135  seconds
1/1 [==============================] - 0s 32ms/step
There are  12  representations found in  

find function lasts  0.7659411430358887  seconds
1/1 [==============================] - 0s 28ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.7315566539764404  seconds
1/1 [==============================] - 0s 27ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.5086169242858887  seconds
1/1 [==============================] - 0s 24ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.7902710437774658  seconds
1/1 [==============================] - 0s 25ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.4698753356933594  seconds
1/1 [==============================] - 0s 24ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.7246999740600586  seconds
1/1 [==============================] - 0s 23ms/step
There are  12  representations found in  rep

find function lasts  0.7845029830932617  seconds
1/1 [==============================] - 0s 31ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.7318730354309082  seconds
1/1 [==============================] - 0s 27ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.7531039714813232  seconds
1/1 [==============================] - 0s 24ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.4587538242340088  seconds
1/1 [==============================] - 0s 30ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.4743161201477051  seconds
1/1 [==============================] - 0s 30ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.5374059677124023  seconds
1/1 [==============================] - 0s 30ms/step
There are  12  representations found in  rep

find function lasts  0.8225398063659668  seconds
1/1 [==============================] - 0s 28ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.8895213603973389  seconds
1/1 [==============================] - 0s 28ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.5241119861602783  seconds
1/1 [==============================] - 0s 28ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.7585549354553223  seconds
1/1 [==============================] - 0s 29ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.742462158203125  seconds
1/1 [==============================] - 0s 29ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.790647029876709  seconds
1/1 [==============================] - 0s 27ms/step
There are  12  representations found in  repre

find function lasts  0.43538403511047363  seconds
1/1 [==============================] - 0s 27ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.49149608612060547  seconds
1/1 [==============================] - 0s 25ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.4557819366455078  seconds
1/1 [==============================] - 0s 28ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.4265310764312744  seconds
1/1 [==============================] - 0s 34ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.4547889232635498  seconds
1/1 [==============================] - 0s 29ms/step
There are  12  representations found in  representations_vgg_face.pkl
find function lasts  0.4791989326477051  seconds
1/1 [==============================] - 0s 29ms/step
There are  12  representations found in  r